In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import rcParams
import time
from copy import copy

#from sim_handling import Simulation,loadSim #uses ds7
from sim_handling2 import Simulation,loadSim #ds8: faster, more space efficient
from diffusionstuff8 import f0d,f1d,f2d, getsigmastep_2d
from matplotlib.animation import PillowWriter

## 1D FFT Analysis of Ice Continuum Simulation

### Overview

1. **Simulation Setup**: Set up a 2D ice surface simulation and parameters.
2. **1D Simulation Creation**: Create a 1D simulation mimicking the 2D sim.
3. **Number of Steps Analysis**: Calculate the number of steps on the ice surface over time and plot the variation.
4. **Fourier Analysis**: Apply FFT to the number of steps signal and visualize frequency-magnitude plots.
5. **Signal Reconstruction**: Reconstruct the original signal from the FFT analysis.
6. **Additional Visualizations**: Display a 3D plot of the 1D surface over time and a sample plot of the ice layers.

This analysis provides insights into the variation and frequency characteristics of the simulated ice surface using the ice continuum model.

Note - currently unneccisarily verbose setup for sims, as only the 1D simulation is used.

In [ ]:
%matplotlib widget

#set figsize to be big (12,12) inches
#rcParams['figure.figsize'] = 10,10

In [ ]:
#set up and run and plot a new simulation
test_2d_asym = Simulation(f2d, (200,600), method='RK45')

#try a noisy simulation
test_2d_asym.noisy_init = False

#test_2d_asym.float_params['DoverdeltaX2'] *= 2 
test_2d_asym.layermax = 4000
#test_2d_asym.sigmastepmax= -0.01 #ablation
test_2d_asym.sigmastepmax= 0.3 #growth
test_2d_asym.center_reduction = 0.1
#test_2d_asym.deltaT *= 10 #increase time step to decrease number of steps saved

#_plot = test_2d_asym.plot(tot=True,completion=1)
#anim = test_2d_asym.animate(ice=True,surface=True, ret=True, focus_on_growth=True)

In [ ]:
#create 1d sim (same args as 2d) and graph number of steps over time
ss_1d = Simulation(f1d, (test_2d_asym.shape[0],), method=test_2d_asym.method, rtol=test_2d_asym.rtol)

ss_1d.float_params['DoverdeltaX2'] = test_2d_asym.float_params['DoverdeltaX2'] 
ss_1d.layermax = test_2d_asym.layermax * 1.5 #make sure it reaches steady state
ss_1d.sigmastepmax = test_2d_asym.sigmastepmax 
ss_1d.sigma0 = test_2d_asym.sigma0
ss_1d.center_reduction = test_2d_asym.center_reduction
ss_1d.deltaT = test_2d_asym.deltaT #increase time step to decrease number of steps saved?
ss_1d.noisy_init = test_2d_asym.noisy_init

In [ ]:
#print number of steps over time
Ntot = ss_1d.getNtot()

maxes = np.array([np.max(n) for n in Ntot[:]])
mins = np.array([np.min(n) for n in Ntot[:]])

#print(maxes,mins)
plt.figure()
ax = plt.subplot(111)

#ax.plot(test_f2d_sim.results()['t'],maxes-mins)
ax.plot(Ntot[:,0],maxes-mins)
ax.set_ylabel('Number of steps on surface')
ax.set_xlabel('Time')
plt.grid()

In [ ]:
#plt.close()
#fourier analysis

xs = Ntot[:,0]
ys = maxes-mins
ys_fft = np.fft.fft(ys)
freq = np.fft.fftfreq(xs.shape[-1])


#plot the fft
#plt.figure('Real and imaginary parts of FFT in freq domain')
plt.figure('fft frequency-magnitude plots of #steps on ice surface over time')
#ax = plt.axes()
ax = plt.subplot(131)
#ax.plot(freq, ys_fft.real, label='real')
#ax.plot(freq, ys_fft.imag, label='imaginary')

#ax.plot(freq, np.sqrt(ys_fft.real**2 + ys_fft.imag**2), label='magnitude')
ax.set_title('frequency-magnitude plot of fft of entire signal')
ax.plot(list(map(lambda x: x*len(freq),freq)), np.sqrt(ys_fft.real**2 + ys_fft.imag**2), label='magnitude')

#ax.plot(freq, np.abs(ys_fft), label='magnitude')
#ax.stem(freq, np.abs(ys_fft), 'b', \
#         markerfmt=" ", basefmt="-b",label='magnitude')

ax.set_xlabel('Frequency')
ax.set_ylabel('Magnitude')


ax = plt.subplot(132)
ax.set_xlabel('Frequency')
ax.set_ylabel('Magnitude')
#ax.set_title('fft before numerical instability')
#plot the fft of just the part before numerical instability\
cutpoint = int(len(ys)*2/5)
ys_pre_num = ys[:cutpoint]
xs_pre_num = xs[:cutpoint]
ys_fft_pre_num = np.fft.fft(ys_pre_num)
freq_pre_num = np.fft.fftfreq(xs_pre_num.shape[-1])
pre = np.sqrt(ys_fft_pre_num.real**2 + ys_fft_pre_num.imag**2)
ax.plot(list(map(lambda x: x*len(freq),freq_pre_num)), pre, label='magnitude pre-instabilty')

#ax = plt.subplot(123)
#ax.set_xlabel('Frequency')
#ax.set_ylabel('Magnitude')
#ax.set_title('fft after numerical instability')
ax.set_title('fft before vs after numerical instability')
#plot the fft of just the part AFTER numerical instability
cutpoint = int(len(ys)*2/5)
ys_post_num = ys[cutpoint:]
xs_post_num = xs[cutpoint:]
ys_fft_post_num = np.fft.fft(ys_post_num)
freq_post_num = np.fft.fftfreq(xs_post_num.shape[-1])
post = np.sqrt(ys_fft_post_num.real**2 + ys_fft_post_num.imag**2)
ax.plot(list(map(lambda x: x*len(freq),freq_post_num)), post, label='magnitude post-instabilty')
ax.legend()

#plot difference of pre and post plots
ax = plt.subplot(133)
ax.set_xlabel('Frequency')
ax.set_ylabel('Magnitude')
ax.set_title('difference of fft before and after numerical instability')
cut = int((len(post)-len(pre))//2)
ax.plot(list(map(lambda x: x*len(freq),freq_pre_num)), post[cut:-cut] - pre)

#NOTE idea: scale pre to post based off magnitude of zero peak so when subtracting, just the new frequencies of the numerical instability are shown (and not base/natural frequencies)

#plot the fft with high frequencies filtered out
# ys_fft_filtered = ys_fft.copy()
# ys_fft_filtered[np.abs(freq) > 0.1] = 0
# ax.plot(freq, np.abs(ys_fft_filtered), label='magnitude filtered')

#ax.stem(freq, np.abs(ys_fft_filtered), 'r', \
#         markerfmt=" ", basefmt="-r",label='magnitude filtered')



In [ ]:
# #fourier analysis filtered to remove lowest freq signals
# #plt.close()

# xs = Ntot[:,0]
# ys = maxes-mins
# ys_fft = np.fft.fft(ys)
# ys_fft_filtered = ys_fft.copy()
# freq = np.fft.fftfreq(xs.shape[-1])
# ys_fft_filtered[np.abs(freq) < 600] = 30 # removes lowest frequency noise


# #plot the fft
# #plt.figure('Real and imaginary parts of FFT in freq domain')
# plt.figure('fft frequency-magnitude plots of #steps on ice surface over time')
# ax = plt.subplot(131)
# ax.set_title('frequency-magnitude plot of fft of entire signal')
# ax.plot(list(map(lambda x: x*len(freq),freq)), np.sqrt(ys_fft_filtered.real**2 + ys_fft_filtered.imag**2), label='magnitude')
# ax.set_xlabel('Frequency')
# ax.set_ylabel('Magnitude')


# ax = plt.subplot(132)
# ax.set_title('fft before vs after numerical instability')
# ax.set_xlabel('Frequency')
# ax.set_ylabel('Magnitude')
# #ax.set_title('fft before numerical instability')
# #plot the fft of just the part before numerical instability\
# cutpoint = int(len(ys)*2/5)
# ys_pre_num = ys[:cutpoint]
# xs_pre_num = xs[:cutpoint]
# ys_fft_pre_num = np.fft.fft(ys_pre_num)
# freq_pre_num = np.fft.fftfreq(xs_pre_num.shape[-1])
# ys_fft_pre_num_filtered = ys_fft_pre_num.copy()
# ys_fft_pre_num_filtered[np.abs(freq_pre_num) < 600] = 30 # removes lowest frequency noise

# pre = np.sqrt(ys_fft_pre_num_filtered.real**2 + ys_fft_pre_num_filtered.imag**2)
# ax.plot(list(map(lambda x: x*len(freq),freq_pre_num)), pre, label='magnitude pre-instabilty')

# #plot the fft of just the part AFTER numerical instability
# cutpoint = int(len(ys)*2/5)
# ys_post_num = ys[cutpoint:]
# xs_post_num = xs[cutpoint:]
# ys_fft_post_num = np.fft.fft(ys_post_num)
# freq_post_num = np.fft.fftfreq(xs_post_num.shape[-1])
# ys_fft_post_num_filtered = ys_fft_post_num.copy()
# ys_fft_post_num_filtered[np.abs(freq_post_num) < 600] = 30 # removes lowest frequency noise

# post = np.sqrt(ys_fft_post_num_filtered.real**2 + ys_fft_post_num_filtered.imag**2)
# ax.plot(list(map(lambda x: x*len(freq),freq_post_num)), post, label='magnitude post-instabilty')
# ax.legend()

# #plot difference of pre and post plots
# ax = plt.subplot(133)
# ax.set_xlabel('Frequency')
# ax.set_ylabel('Magnitude')
# ax.set_title('difference of fft before and after numerical instability')
# cut = int((len(post)-len(pre))//2)
# ax.plot(list(map(lambda x: x*len(freq),freq_pre_num)), post[cut:-cut] - pre)

# #NOTE idea: scale pre to post based off magnitude of zero peak so when subtracting, just the new frequencies of the numerical instability are shown (and not base/natural frequencies)

# #plot the fft with high frequencies filtered out
# # ys_fft_filtered = ys_fft.copy()
# # ys_fft_filtered[np.abs(freq) > 0.1] = 0
# # ax.plot(freq, np.abs(ys_fft_filtered), label='magnitude filtered')

# #ax.stem(freq, np.abs(ys_fft_filtered), 'r', \
# #         markerfmt=" ", basefmt="-r",label='magnitude filtered')



In [ ]:
#calculate dominant frequencies

num_largest_signals = 40
num_decimals_to_round_to = 1
#fft pre_num
#code explained: get the frequencies of the largest signals, only keep unique magnitudes, rounded to nearest 10th scaled by len(freq) to get the actual frequency (units of inverse timestep)
dominant_freqs_pre_num = list(set(map(lambda x: round(x,num_decimals_to_round_to) ,np.abs(freq[np.argsort(np.abs(ys_fft_pre_num))[-num_largest_signals:]])*len(freq))))
#fft post_num
dominant_freqs_post_num = list(set(map(lambda x: round(x,num_decimals_to_round_to) ,np.abs(freq[np.argsort(np.abs(ys_fft_post_num))[-num_largest_signals:]])*len(freq))))
#fft all
dominant_freqs = list(set(map(lambda x: round(x,num_decimals_to_round_to) ,np.abs(freq[np.argsort(np.abs(ys_fft))[-num_largest_signals:]])*len(freq))))

print('Dominant frequencies:', *dominant_freqs)
print('Dominant frequencies pre-numerical instability:', *dominant_freqs_pre_num)
print('Dominant frequencies post-numerical instability:', *dominant_freqs_post_num)
print('Frequencies that appear in pre and post:', *list(set(dominant_freqs_pre_num).intersection(dominant_freqs_post_num)))
print('Dominant freqs. in pre but not post: ', *list(set(dominant_freqs_pre_num).difference(dominant_freqs_post_num)))
print('Dominant freqs. in post but not pre: ', *list(set(dominant_freqs_post_num).difference(dominant_freqs_pre_num)))

In [ ]:
#plot the signals reconstructed from fft
plt.figure('reconstructed signal')
ax = plt.axes()
ax.plot(xs, np.fft.ifft(ys_fft), label='original')  
ax.plot(xs_pre_num, np.fft.ifft(ys_fft_pre_num), label='original pre-instability')
ax.legend()
ax.set_xlabel('x')
ax.set_ylabel('f(x)')

In [ ]:
plt.figure('1d surface over time as 3d plot')
#plot 1d surface over time as 3d plot
Ntot = ss_1d.getNtot()
ax = plt.axes(projection='3d')
#start and stop points in time
startpoint = int(len(Ntot)*5)
cutpoint = int(len(Ntot)*2/5)
xs, ys = np.meshgrid(np.linspace(0,50,100), Ntot[startpoint:cutpoint,0])
ax.plot_surface(xs,ys, Ntot[startpoint:cutpoint,:])

In [ ]:
plt.close()
plt.figure('sample 1d surface')
ax = plt.axes()
Ntots = ss_1d.getNtot()
print('number of timesteps in Ntots',len(Ntots))
ax.plot(ss_1d.x, Ntots[-1] - (np.mean(Ntots[-1]-Ntots[-1000])), 'b-+', label='ice+QLL -1')
ax.plot(ss_1d.x, Ntots[-1000], 'r-+', label='ice+QLL -1000')
ax.set_xlabel(r'x ($\mu m$)')
ax.set_ylabel('Layers of ice')
ax.legend()